In [69]:
#!pip install requests colorlog PyHive mlrun kubernetes


In [70]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')




Project path: /User/parquez
Project name: parquez


In [71]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

In [72]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [73]:
# project.set_function("parquez.py", 'parquezrun', kind='job', image='aviaigz/parquez')

In [74]:
project.set_function("validate-input.py", 'validate', kind='job', image='aviaigz/parquez')

In [75]:
from mlrun import run_local, mount_v3io

In [76]:
# project.func('parquezrun').apply(mount_v3io())
# project.func('parquezrun').set_env('PYTHONPATH', project_path)
# project.func('parquezrun').spec.service_account='mlrun-api'
project.func('validate').apply(mount_v3io())
project.func('validate').set_env('PYTHONPATH', project_path)
project.func('validate').spec.artifact_path = 'User/artifacts'


In [77]:
project.set_function("get_table_schema.py", 'get_schema', kind='job', image='aviaigz/parquez')


In [78]:
project.func('get_schema').apply(mount_v3io())
project.func('get_schema').set_env('PYTHONPATH', project_path)
project.func('get_schema').spec.artifact_path = 'User/artifacts'

In [79]:
project.set_function("create_parquet_table.py", 'create_parquet', kind='job', image='aviaigz/parquez')

In [80]:
project.func('create_parquet').apply(mount_v3io())
project.func('create_parquet').set_env('PYTHONPATH', project_path)
project.func('create_parquet').spec.artifact_path = 'User/artifacts'
project.func('create_parquet').spec.service_account='mlrun-api'

In [81]:
project.set_function("create_kv_view.py", 'create_kv_view', kind='job', image='aviaigz/parquez')

In [82]:
project.func('create_kv_view').apply(mount_v3io())
project.func('create_kv_view').set_env('PYTHONPATH', project_path)
project.func('create_kv_view').spec.artifact_path = 'User/artifacts'

In [83]:
project.set_function("create_unified_view.py", 'create_unified_view', kind='job', image='aviaigz/parquez')

In [84]:
project.func('create_unified_view').apply(mount_v3io())
project.func('create_unified_view').set_env('PYTHONPATH', project_path)
project.func('create_unified_view').spec.artifact_path = 'User/artifacts'

In [85]:
artifact_path = '/User/artifacts'
# #project.func('parquezrun').run()
project.func('validate').run( params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'},artifact_path=artifact_path)

[mlrun] 2020-06-28 11:53:12,329 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:12,330 starting run validate uid=7bcd8133b1e5400fafe8655d61141b0c  -> http://mlrun-api:8080
[mlrun] 2020-06-28 11:53:12,429 Job is running in the background, pod: validate-q42rr
[mlrun] 2020-06-28 11:53:18,554 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:18,564 starting local run: main.py # main
[mlrun] 2020-06-28 11:53:20,099 logging run results to: http://mlrun-api:8080
[mlrun] 2020-06-28 11:53:20,106 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:20,116 {'view_name': 'view_name', 'partition_by': 'h', 'partition_interval': '1h', 'real_time_window': '1d', 'historical_retention': '7d', 'real_time_table_name': 'faker', 'config_path': '/User/parquez/config/parquez.ini'}
[mlrun] 2020-06-28 11:53:20,116 Starting to Parquezzzzzzzz
[mlrun] 2020-06-28 11:53:20,117 Namespace(config='/User/parquez/config/p

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
parquez,...61141b0c,0,Jun 28 11:53:20,completed,validate,host=validate-q42rrkind=jobowner=adminv3io_user=admin,,config_path=/User/parquez/config/parquez.inihistorical_retention=7dpartition_by=hpartition_interval=1hreal_time_table_name=fakerreal_time_window=1dview_name=view_name,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7bcd8133b1e5400fafe8655d61141b0c --project parquez , !mlrun logs 7bcd8133b1e5400fafe8655d61141b0c --project parquez
[mlrun] 2020-06-28 11:53:21,598 run executed, status=completed


In [86]:
# Run the data-ingestion function locally in Jupyter Notebook
#get_data_run = run_local(command= project.func('parquezrun'),workdir='./')

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [87]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io

funcs = {}
parquez_params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'}


# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())

    #functions['serving'].metadata.name = 'getting-started-serving'


# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "parquez-pipeline",
    description = "parquez description"
)
def kfpipeline():
    # Ingest the data set
    validate = funcs['validate'].as_step(
        name="validate",
        params=parquez_params
        #handler='get_data',
        #inputs={'source_url': source_url},
        #params={'format': 'pq'},
        #outputs=[DATASET]
    )
    
    # Analyze the dataset
    schema = funcs['get_schema'].as_step(
        name="get_schema",
        params = parquez_params,
        inputs={'table': validate.outputs},
        #handler='get_data',                
        outputs=['schema']
    )
    
    parquet = funcs["create_parquet"].as_step(
        name="create_parquet",
        params=parquez_params,
        inputs={"table": schema.outputs['schema']})
    
    kv_view = funcs["create_kv_view"].as_step(
        name="create_kv_view",
        params=parquez_params,
        inputs={'table': parquet.outputs}
    )
    
    unified_view = funcs["create_unified_view"].as_step(
        name="create_unified_view",
        params=parquez_params,
        inputs={'table': kv_view.outputs}
    )    
    

#     # Train a model with hyperparemeters
#     train = funcs["train"].as_step(
#         name="train",
#         params={"sample": -1,
#                 "label_column": LABELS,
#                 "test_size": 0.10},
#         hyperparams={'model_pkg_class': MODELS},
#         selector='max.accuracy',
#         inputs={"dataset": ingest.outputs[DATASET]},
#         outputs=['model', 'test_set'])

#     # Test and visualize the model
#     test = funcs["test"].as_step(
#         name="test",
#         params={"label_column": LABELS},
#         inputs={"models_path": train.outputs['model'],
#                 "test_set": train.outputs['test_set']})

#     # Deploy the model as a serverless function
#     deploy = funcs["serving"].deploy_step(
#         models={f"{DATASET}_v1": train.outputs['model']})

#     # Test the new model server (via REST API calls)
#     tester = funcs["serving-tester"].as_step(
#         name='serving-tester',
#         params={'addr': deploy.outputs['endpoint'], 'model': f"{DATASET}_v1"},
#         inputs={'table': train.outputs['test_set']})

Overwriting /User/parquez/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [88]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [89]:
project.save()

In [90]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

[mlrun] 2020-06-28 11:53:21,706 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:21,741 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:21,785 warning!, server (0.4.7) and client (0.4.9) ver dont match
[mlrun] 2020-06-28 11:53:21,822 warning!, server (0.4.7) and client (0.4.9) ver dont match


[mlrun] 2020-06-28 11:53:21,956 Pipeline run id=11c75e55-922c-4ea9-80b3-3c6af4a702da, check UI or DB for progress
